In [2]:
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill
from datetime import datetime, timedelta
import pandas as pd

def allocate_courses(courses_df, venues_df, cbt_venues_df):
    """Allocate courses to venues and time slots while adhering to constraints."""
    # Define time slots
    time_slots = [
        {"Session": "Morning", "Start": "08:00 AM", "End": "10:30 AM"},
        {"Session": "Afternoon", "Start": "11:00 AM", "End": "01:00 PM"},
        {"Session": "Evening", "Start": "02:00 PM", "End": "04:30 PM"}
    ]

    # Define two weeks of dates (Monday-Saturday + Saturday overflow)
    start_date = datetime.strptime("2024-04-01", "%Y-%m-%d")  # Adjust as needed
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    dates = []
    for i in range(14):  # Two weeks
        current_date = start_date + timedelta(days=i)
        if current_date.strftime("%A") in days:
            dates.append(current_date)

    # Prepare data structures for scheduling
    cbt_allocated = []
    written_allocated = []
    unscheduled = []
    invigilator_counter = 1  # To assign unique invigilator IDs

    # Venue availability
    venue_availability = {
        venue: {date: {slot["Session"]: {"Used": False, "Capacity": cap} for slot in time_slots}
                for date in dates}
        for venue, cap in zip(venues_df["Venue"], venues_df["Capacity"])
    }
    cbt_venue_availability = {
        venue: {date: {slot["Session"]: {"Used": False, "Capacity": cap} for slot in time_slots}
                for date in dates}
        for venue, cap in zip(cbt_venues_df["Venue"], cbt_venues_df["Capacity"])
    }

    # Schedule courses
    for _, course in courses_df.iterrows():
        scheduled = False
        student_count = course["Number of Students"]
        exam_type = str(course["Exam Type"]).strip().lower()
        course_code = course["Course Code"]

        # Choose venues based on exam type
        venues_to_check = cbt_venue_availability if exam_type == "cbt" else venue_availability

        # Loop through dates and sessions to schedule exams
        for date in dates:
            day_name = date.strftime("%A")
            for slot in time_slots:
                if exam_type == "cbt":
                    # CBT exam: Check if there is enough combined capacity across venues
                    total_capacity = 0
                    selected_venues = []
                    for venue, slots in venues_to_check.items():
                        if not slots[date][slot["Session"]]["Used"]:
                            total_capacity += slots[date][slot["Session"]]["Capacity"]
                            selected_venues.append(venue)
                        if total_capacity >= student_count:
                            break

                    if total_capacity >= student_count:
                        # Assign invigilators for CBT exams
                        invigilators = [
                            f"C1-{invigilator_counter}", f"L1-{invigilator_counter+1}",
                            f"L2-{invigilator_counter+2}", f"L3-{invigilator_counter+3}",
                            f"L4-{invigilator_counter+4}", f"E-{invigilator_counter+5}"
                        ]
                        invigilator_counter += 6
                        cbt_allocated.append({
                            "Date": date.strftime("%Y-%m-%d"),
                            "Day": day_name,
                            "Time": f"{slot['Start']} - {slot['End']}",
                            "Course": course_code,
                            "Programme": course["Program"],
                            "Invigilators": ", ".join(invigilators),
                            "Venues": ", ".join(selected_venues)
                        })
                        # Mark selected venues as used
                        for venue in selected_venues:
                            venues_to_check[venue][date][slot["Session"]]["Used"] = True
                        scheduled = True
                        break

                else:
                    # Written exam: Check for available venue
                    for venue, slots in venues_to_check.items():
                        if not slots[date][slot["Session"]]["Used"] and student_count <= slots[date][slot["Session"]]["Capacity"]:
                            # Assign invigilators for written exams (1 per 40 students, min 2)
                            invigilators = max(2, (student_count // 40) + 1)
                            written_allocated.append({
                                "Date": date.strftime("%Y-%m-%d"),
                                "Day": day_name,
                                "Session": slot["Session"],
                                "Start": slot["Start"],
                                "End": slot["End"],
                                "Course Code": course_code,
                                "Course Name": course["Program"],
                                "Venue": venue,
                                "Invigilators": invigilators,
                                "Number of Students": student_count
                            })
                            # Mark the venue as used
                            slots[date][slot["Session"]]["Used"] = True
                            scheduled = True
                            break

                if scheduled:
                    break
            if scheduled:
                break

        if not scheduled:
            # Track unscheduled courses
            unscheduled.append({
                "Course Code": course_code,
                "Program": course["Program"],
                "Exam Type": exam_type.capitalize(),
                "Number of Students": student_count,
                "Reason": "No available time slots or venue capacity"
            })

    return pd.DataFrame(cbt_allocated), pd.DataFrame(written_allocated), pd.DataFrame(unscheduled)

def write_timetable(output_file, cbt_df, written_df, unscheduled_df):
    """Write the allocated and unscheduled courses to an Excel file."""
    workbook = Workbook()

    # CBT Exams Sheet
    cbt_sheet = workbook.active
    cbt_sheet.title = "CBT Exams"
    cbt_headers = ["Date", "Day", "Time", "Course", "Programme", "Invigilators", "Venues"]
    cbt_sheet.append(cbt_headers)
    for _, row in cbt_df.iterrows():
        cbt_sheet.append(list(row))

    # Written Exams Sheet
    written_sheet = workbook.create_sheet("Written Exams")
    written_headers = ["Date", "Day", "Session", "Start", "End", "Course Code", "Course Name",
                       "Venue", "Invigilators", "Number of Students"]
    written_sheet.append(written_headers)
    for _, row in written_df.iterrows():
        written_sheet.append(list(row))

    # Unscheduled Courses Sheet
    unscheduled_sheet = workbook.create_sheet("Unscheduled Courses")
    unscheduled_headers = ["Course Code", "Program", "Exam Type", "Number of Students", "Reason"]
    unscheduled_sheet.append(unscheduled_headers)
    for _, row in unscheduled_df.iterrows():
        unscheduled_sheet.append(list(row))

    # Apply text wrapping and adjust column widths
    for sheet in [cbt_sheet, written_sheet]:
        for row in sheet.iter_rows():
            for cell in row:
                cell.alignment = Alignment(wrap_text=True)
        
        for col in sheet.columns:
            max_length = max((len(str(cell.value)) for cell in col if cell.value), default=0)
            adjusted_width = min(40, max(10, max_length + 2))  # Adjust column width between 10 and 40
            sheet.column_dimensions[col[0].column_letter].width = adjusted_width

    # Save workbook
    workbook.save(output_file)
    print(f"Timetable saved to {output_file}")

def main(input_file, output_file):
    # Load the processed data
    courses_df = pd.read_excel(input_file, sheet_name="Courses")
    venues_df = pd.read_excel(input_file, sheet_name="Venues")
    cbt_venues_df = pd.read_excel(input_file, sheet_name="CBT Venues")

    # Allocate courses to slots and venues
    cbt_df, written_df, unscheduled_df = allocate_courses(courses_df, venues_df, cbt_venues_df)

    # Write the results to the output file
    write_timetable(output_file, cbt_df, written_df, unscheduled_df)

if __name__ == "__main__":
    input_excel = "datam10.xlsx"  # Input file from Script 1
    output_excel = "timetablem107.xlsx"  # Output timetable file
    main(input_excel, output_excel)


Timetable saved to timetablem107.xlsx
